In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
tfd = tfp.distributions

In [2]:
tf.__version__

'2.1.0'

In [3]:
# Define Bayes Model
def bayes_model_arch(voxel_dim=64,deviation_channels=3,output_dimension=6):
    kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) / tf.cast(4000, dtype=tf.float32))
    negloglik = lambda y, rv_y: -rv_y.log_prob(y)
    aleatoric_std=0.001
    aleatoric_tensor=[aleatoric_std] * output_dimension
    model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(voxel_dim,voxel_dim,voxel_dim,deviation_channels)),
    tfp.layers.Convolution3DFlipout(32, kernel_size=(5,5,5),kernel_divergence_fn=kl_divergence_function,strides=(2,2,2),activation=tf.nn.relu),
    tfp.layers.Convolution3DFlipout(32, kernel_size=(4,4,4),kernel_divergence_fn=kl_divergence_function,strides=(2,2,2),activation=tf.nn.relu),
    tfp.layers.Convolution3DFlipout(32, kernel_size=(3,3,3),kernel_divergence_fn=kl_divergence_function,strides=(1,1,1),activation=tf.nn.relu),
    tf.keras.layers.MaxPooling3D(pool_size=[2, 2, 2]),
    tf.keras.layers.Flatten(),
    tfp.layers.DenseFlipout(128,activation=tf.nn.relu,kernel_divergence_fn=kl_divergence_function),
    tfp.layers.DenseFlipout(64,kernel_divergence_fn=kl_divergence_function,activation=tf.nn.relu),
    tfp.layers.DenseFlipout(output_dimension,kernel_divergence_fn=kl_divergence_function),
    tfp.layers.DistributionLambda(lambda t:tfd.MultivariateNormalDiag(loc=t[..., :output_dimension], scale_diag=aleatoric_tensor)),])

    #negloglik = lambda y, p_y: -p_y.log_prob(y)
    #experimental_run_tf_function=False
    #tf.keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),experimental_run_tf_function=False,loss=negloglik,metrics=[tf.keras.metrics.MeanAbsoluteError()])
    #print("3D CNN model successfully compiled")
    print(model.summary())
    return model

model=bayes_model_arch()

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_flipout (Conv3DFlipou (None, 30, 30, 30, 32)    24032     
_________________________________________________________________
conv3d_flipout_1 (Conv3DFlip (None, 14, 14, 14, 32)    131104    
_________________________________________________________________
conv3d_flipout_2 (Conv3DFlip (None, 12, 12, 12, 32)    55328     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 6, 6, 6, 32)       0         
_________________________________________________________________
flatten (Flatten)            (None, 6912)              0         
_________________________________________________________________
dense_flipout (DenseFlipout)

In [4]:
#Loading Model Weights
part_type="inner_rf_assembly"
train_path='../trained_models/'+part_type
weight_path=train_path+'/model'+'/Bayes_trained_model_0'
model.load_weights(weight_path)

In [5]:
# Template for extracting weights [0] mean weights , [1] Std Dev Weights ,[2] Bias (deterministic)
W_Input_Hidden = model.layers[0].get_weights()
len(W_Input_Hidden)
W_Input_Hidden[2].shape

(32,)

In [6]:
from lr_multiplier import LearningRateMultiplier